# Main breakthroughs in Word embeddings, from Word Counts to LLMs:

**Low-tech beginnings:**

* **Word Counts:** Basic frequency of words in a corpus served as initial representations. Useful for simple tasks but ignored semantic relationships.
* **Term Frequency (TF):** simply counting the occurrences of words within a corpus was one of the earliest methods for representing words. This basic approach laid the foundation for more sophisticated techniques.
* **N-grams:** These are sequences of n consecutive words (e.g., bigrams for pairs, trigrams for triplets). By analyzing n-gram frequencies, we can capture some local context and word relationships, going beyond individual word counts.


**Distributional Semantics:**

* **Word2Vec (2013):** First major breakthrough. Learned word embeddings by predicting surrounding words, capturing semantic similarities.
* **GloVe (2014):** Leverages co-occurrence statistics for better context sensitivity.
* **FastText (2016):** Incorporates subword information, handling rare words and morphological variants.

**Contextual Embeddings:**

* **ELMo (2018):** Uses bi-directional LSTMs to capture word meaning based on surrounding context.
* **BERT (2018):** Pre-trained transformer model on large unlabeled text, learning contextualized representations.
* **XLNet (2019):** Builds upon BERT's masked language modeling with permutation language modeling for better understanding of word relationships.

**Towards Understanding and Generation:**

* **GPT-3 (2020):** Generative Pre-trained Transformer 3, a large language model (LLM) with impressive text generation capabilities.
* **LaMDA (2021):** Language Model for Dialogue Applications, focuses on factual consistency and grounding in conversation.
* **PaLM (2022):** Pathways Language Model, pushes the boundaries of LLM size and performance, demonstrating progress in reasoning and question answering.

# Import Libraries

In [1]:
import pandas as pd
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist, MLEProbDist
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import keras
from scipy.sparse.linalg import svds
from sklearn.decomposition import TruncatedSVD

**Define Documents list**

In [2]:
Doc_1= "The cat in the hat"
Doc_2= "The quick brown fox"
Doc_3= "The hat is blue"

Docs =[Doc_1,Doc_2,Doc_3]

# **Term Frequency (TF)**
* **TF(t,d) is the term frequency of term t in document d (how often the term appears in the document).*** **TF(t,d) is the term frequency of term t in document d (how often the term appears in the document).**

In [3]:
#get distinct words for each document
lst = []
for d in Docs:
    lst.extend(d.lower().split(' '))
wrds = set(lst) # remove duplicate words
wrds

{'blue', 'brown', 'cat', 'fox', 'hat', 'in', 'is', 'quick', 'the'}

In [4]:
#form a dataframe to represent TF for each word in each Document where columns are words and rows are documents
def count_wrd_Doc(wrd,doc):
    i=0
    for w in doc.lower().split(' '):
        if wrd == w:
            i = i+1
    return i/len(doc.lower().split(' '))
    
tf_df = pd.DataFrame(columns=list(wrds)) #empty dataframe initialized with words column headers
freq_lst=[] #empty list for each column to save word frequencies in each document
for c in tf_df.columns:
    freq_lst=[]#empty the list
    for d in Docs:
        freq_lst.append(count_wrd_Doc(c,d))#append the frequency of word in document d
    tf_df[c]=freq_lst #assign values to column
tf_df #display the dataframe of TF for each word in each document

,in,is,the,fox,quick,brown,blue,cat,hat
0,0.2,0.00,0.40,0.00,0.00,0.00,0.00,0.2,0.20
1,0.0,0.00,0.25,0.25,0.25,0.25,0.00,0.0,0.00
2,0.0,0.25,0.25,0.00,0.00,0.00,0.25,0.0,0.25


**Document Frequency (DF)**
* **Calculate Document Frequency (DF): the word appears in how many documents*** **Calculate Document Frequency (DF): the word appears in how many documents**

In [5]:
df_df = pd.DataFrame(columns=list(wrds)) #empty dataframe initialized with words column headers
for c in df_df.columns:
    df_df[c] = [sum(1 for doc in Docs if c in doc.lower().split(' '))]
df_df #display the dataframe of DF for each word 

,in,is,the,fox,quick,brown,blue,cat,hat
0,1,1,3,1,1,1,1,1,2


**Inverse Document Frequency (IDF)**
* **IDF(t,D) is the inverse document frequency of term t in the entire document set D (logarithmically scaled inverse fraction of the documents that contain the term).*** **IDF(t,D) is the inverse document frequency of term t in the entire document set D (logarithmically scaled inverse fraction of the documents that contain the term).**

In [6]:
idf_df = pd.DataFrame(columns=list(wrds)) #empty dataframe initialized with words column headers
for c in idf_df.columns:
    N = 3 #No of documents
    df = df_df[c].iloc[0] # DF of word
    idf_df[c] = [math.log((N+1) / (df+1))+1]#IDF = log (no. of documents/DF(word)) 
idf_df #display the dataframe of idf for each word 

,in,is,the,fox,quick,brown,blue,cat,hat
0,1.693147,1.693147,1.0,1.693147,1.693147,1.693147,1.693147,1.693147,1.287682


**Term Frequency - Inverse Document Frequency (TF-IDF)= TF * IDF**

In [7]:
tfidf_df = pd.DataFrame(columns=list(wrds)) #empty dataframe initialized with words column headers
tfidf_lst=[]  #empty list for each column 
for c in tfidf_df.columns:
    tfidf_lst=[] #empty list for each column
    for i in range(0,len(Docs)):
        tf_idf_d1 = tf_df[c].iloc[i]*idf_df[c].iloc[0] #append tf of word in i th document to idf of word
        tfidf_lst.append(tf_idf_d1)
    tfidf_df[c]=tfidf_lst#assign tfidf values for each word
tfidf_df #display the dataframe of tf-idf for all words

,in,is,the,fox,quick,brown,blue,cat,hat
0,0.338629,0.000000,0.40,0.000000,0.000000,0.000000,0.000000,0.338629,0.257536
1,0.000000,0.000000,0.25,0.423287,0.423287,0.423287,0.000000,0.000000,0.000000
2,0.000000,0.423287,0.25,0.000000,0.000000,0.000000,0.423287,0.000000,0.321921


**L2 Normalization**
* **L2 normalization, also known as Euclidean normalization or L2 norm normalization, is a technique used to scale vectors (or arrays) in such a way that their Euclidean norm becomes equal to 1.**

In [8]:
normalized_df = pd.DataFrame(columns=tfidf_df.columns)

# Apply L2 normalization to each document's TF-IDF values
for i,row in enumerate(tfidf_df.iterrows()):
    # Extract TF-IDF values    
    tfidf_values_list = list(tfidf_df.iloc[i].values)
    # Calculate L2 norm
    l2_norm = math.sqrt(sum(val**2 for val in tfidf_values_list))
    # Normalize TF-IDF values using L2 norm
    normalized_tfidf = [val / l2_norm for val in list(tfidf_df.iloc[i].values)]
    new_row = pd.Series(normalized_tfidf, index=tfidf_df.columns)
    normalized_df.loc[len(normalized_df)] = new_row
    
normalized_df

,in,is,the,fox,quick,brown,blue,cat,hat
0,0.501651,0.000000,0.592567,0.000000,0.000000,0.000000,0.000000,0.501651,0.381519
1,0.000000,0.000000,0.322745,0.546454,0.546454,0.546454,0.000000,0.000000,0.000000
2,0.000000,0.584483,0.345205,0.000000,0.000000,0.000000,0.584483,0.000000,0.444514


**TfidfVectorizer Python Library**

In [9]:
# Create the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(norm='l2',smooth_idf=True)

# Fit the documents and transform them into a TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(Docs)

# Get the feature names (terms) from the vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()

df_tfidf = pd.DataFrame(data=tfidf_matrix.toarray(), columns=feature_names)
df_tfidf

,blue,brown,cat,fox,hat,in,is,quick,the
0,0.000000,0.000000,0.501651,0.000000,0.381519,0.501651,0.000000,0.000000,0.592567
1,0.000000,0.546454,0.000000,0.546454,0.000000,0.000000,0.000000,0.546454,0.322745
2,0.584483,0.000000,0.000000,0.000000,0.444514,0.000000,0.584483,0.000000,0.345205


* **The TfidfVectorizer in scikit-learn, by default, adds a smoothing term to the denominator of the IDF calculation to avoid division by zero. This is done to handle the case where a term is present in all documents, ensuring that the IDF is not undefined.**
* **L2 normalization, also known as Euclidean normalization or L2 norm normalization, is a technique used to scale vectors (or arrays) in such a way that their Euclidean norm becomes equal to 1.**

# Unigram
* **A unigram, in the context of natural language processing (NLP) and linguistics, refers to a single unit or token of a word. It is the simplest form of linguistic analysis where text is broken down into individual words. In other words, a unigram is a term used to describe a single word in a sequence of words.**
* **Unigrams are the building blocks for more complex linguistic analyses, such as bigrams (pairs of consecutive words), trigrams (triplets of consecutive words), and n-grams in general.*** **A unigram, in the context of natural language processing (NLP) and linguistics, refers to a single unit or token of a word. It is the simplest form of linguistic analysis where text is broken down into individual words. In other words, a unigram is a term used to describe a single word in a sequence of words.**
* **Unigrams are the building blocks for more complex linguistic analyses, such as bigrams (pairs of consecutive words), trigrams (triplets of consecutive words), and n-grams in general.**

In [10]:
#Probability of unigram P(w)=C(w)/m same idea of TF
def count_wrd_Doc(wrd,doc):
    i=0
    for w in doc.lower().split(' '):
        if wrd == w:
            i = i+1
    return i
    
unigram_df = pd.DataFrame(columns=list(wrds)) #empty dataframe initialized with words column headers
freq_lst=[] #empty list for each column to save word frequencies in each document
for c in tf_df.columns:
    freq_lst=[]#empty the list
    for d in Docs:
        freq_lst.append(count_wrd_Doc(c,d))#append the frequency of word in document d
    unigram_df[c]=freq_lst #assign values to column
unigram_df #display the dataframe of TF for each word in each document

,in,is,the,fox,quick,brown,blue,cat,hat
0,1,0,2,0,0,0,0,1,1
1,0,0,1,1,1,1,0,0,0
2,0,1,1,0,0,0,1,0,1


**Unigrams python function**

In [11]:
for d in Docs:
    words = word_tokenize(d.lower())
    result = list(ngrams(words, 1))
    # Calculate frequency distribution of bigrams
    ngram_freq = FreqDist(result)
    for word, frequency in ngram_freq.items():
        print(f"{word}: {frequency}")

('the',): 2
('cat',): 1
('in',): 1
('hat',): 1
('the',): 1
('quick',): 1
('brown',): 1
('fox',): 1
('the',): 1
('hat',): 1
('is',): 1
('blue',): 1


**Bigram**
* **A bigram, in the context of natural language processing (NLP) and linguistics, refers to an ordered pair of consecutive words within a text or sequence of words. It is a type of n-gram, where "n" represents the number of words in the sequence.*** **A bigram, in the context of natural language processing (NLP) and linguistics, refers to an ordered pair of consecutive words within a text or sequence of words. It is a type of n-gram, where "n" represents the number of words in the sequence.**

In [12]:
#get bi-grams of input sentence
def bi_lst(doc):
    wrds = doc.lower().split(' ')
    bi_lst = []
    for j in range(0,len(wrds)-1):
        bi_lst.append(wrds[j:j+2])
    return bi_lst

lst = []
for d in Docs:
    lst.extend(bi_lst(d))
unique_list = []
unique_list = [item for item in lst if item not in unique_list]

def count_biwrd_Doc(st,doc):
    i=0    
    for s in bi_lst(doc):
        if s == st.split(' '):
            i = i+1
    return i
bigram_df = pd.DataFrame(columns=list((' '.join(x) for x in unique_list))) #empty dataframe initialized with words column headers
freq_lst=[] #empty list for each column to save word frequencies in each document
for c in bigram_df.columns:
    freq_lst=[]#empty the list
    for d in Docs:
        freq_lst.append(count_biwrd_Doc(c,d))#append the frequency of word in document d
    bigram_df[c]=freq_lst #assign values to column
bigram_df #display the dataframe of TF for each word in each document

,the cat,cat in,in the,the hat,the quick,quick brown,brown fox,the hat,hat is,is blue
0,1,1,1,1,0,0,0,1,0,0
1,0,0,0,0,1,1,1,0,0,0
2,0,0,0,1,0,0,0,1,1,1


**Bigrams python function**

In [13]:
for d in Docs:
    words = word_tokenize(d.lower())
    result = list(ngrams(words, 2))
    # Calculate frequency distribution of bigrams
    ngram_freq = FreqDist(result)
    for word, frequency in ngram_freq.items():
        print(f"{word}: {frequency}")

('the', 'cat'): 1
('cat', 'in'): 1
('in', 'the'): 1
('the', 'hat'): 1
('the', 'quick'): 1
('quick', 'brown'): 1
('brown', 'fox'): 1
('the', 'hat'): 1
('hat', 'is'): 1
('is', 'blue'): 1


# Skip-gram & Continuous Bag of Words (CBOW)
* These models use shallow neural networks to learn word embeddings. Skip-gram predicts context words given a target word, while CBOW predicts a target word given its context. These models use shallow neural networks to learn word embeddings. Skip-gram predicts context words given a target word, while CBOW predicts a target word given its context. 

**CBOW**
* Objective: The main objective of CBOW is to predict a target word given its context (surrounding words). It learns to represent words in a continuous vector space based on their distributional semantics.
* Architecture: CBOW uses a neural network with a single hidden layer. The input layer and output layer are typically equal to the size of the vocabulary, and the hidden layer has a much smaller dimension, often referred to as the embedding dimension.
* Input and Output: The input to the CBOW model is a set of context words represented as one-hot vectors (binary vectors with a 1 at the index corresponding to the word's position in the vocabulary). The output is the target word's one-hot vector.
* Context Window: The context window is a fixed-size window of surrounding words used to predict the target word. The model is trained to predict the target word based on the words within this context window.

In [14]:
def count_wrd_Doc(wrd,doc):
    i=0
    for w in doc.lower().split(' '):
        if wrd == w:
            i = i+1
    return i
    
cw_df = pd.DataFrame(columns=list(wrds)) #empty dataframe initialized with words column headers
freq_lst=[] #empty list for each column to save word frequencies in each document
for c in tf_df.columns:
    freq_lst=[]#empty the list
    for d in Docs:
        freq_lst.append(count_wrd_Doc(c,d))#append the frequency of word in document d
    cw_df[c]=freq_lst #assign values to column
cw_df #display the datafram

,in,is,the,fox,quick,brown,blue,cat,hat
0,1,0,2,0,0,0,0,1,1
1,0,0,1,1,1,1,0,0,0
2,0,1,1,0,0,0,1,0,1


In [15]:
# Create a CountVectorizer instance
vectorizer = CountVectorizer()

# Fit and transform the documents to create the Bag of Words representation
X_bow = vectorizer.fit_transform(Docs)
feature_names = vectorizer.get_feature_names_out()
# Print the Bag of Words representation
print("Bag of Words representation:")
print(X_bow.toarray())
print("Feature names:")
print(feature_names)

Bag of Words representation:
[[0 0 1 0 1 1 0 0 2]
 [0 1 0 1 0 0 0 1 1]
 [1 0 0 0 1 0 1 0 1]]
Feature names:
['blue' 'brown' 'cat' 'fox' 'hat' 'in' 'is' 'quick' 'the']


**Skip-Grams**
* **Objective:** The main objective of the Skip-gram model is to learn distributed representations (word embeddings) of words in a continuous vector space. It does so by predicting the context words based on a given target word.
* **Architecture:** Skip-gram uses a neural network with a single hidden layer. The input layer and output layer are typically equal to the size of the vocabulary, and the hidden layer has a much smaller dimension, often referred to as the embedding dimension.
* **Input and Output:** The input to the Skip-gram model is a one-hot vector representing a target word (the word for which embeddings are being learned). The output is a probability distribution over the vocabulary, representing the likelihood of each word being a context word.
* **Context Window:** During training, a context window is defined around the target word. The context words within this window are used to predict the target word. The context window provides local context information for each target word.
* **Training Objective:** Skip-gram is trained using a supervised learning approach. The model aims to minimize the cross-entropy loss between the predicted probability distribution over the vocabulary and the actual distribution (one-hot vector of the true context word).

In [16]:
# Generate training pairs (target word, context word)
window_size = 3# the window specifies the context words size neigbored the target word 
training_pairs = []
context_words = []#the context words for w=3 i-3,i-2,i-1,i+1,i+2,i+3
for d in Docs:
    t=[]
    c=[]
    for i, target_word in enumerate(d.lower().split(' ')):
        start = max(0, i - window_size)
        end = min(len(d.lower().split(' ')), i + window_size + 1)
        c = [d.lower().split(' ')[j] for j in range(start, end) if j != i]
        for context_word in c:
            t.append((target_word, context_word))
        print("document:",d.lower())
        print("target:",target_word)
        print("context words",c,",window_size",window_size)
        print("----------------------------------------")
    training_pairs.append(t)    

document: the cat in the hat
target: the
context words ['cat', 'in', 'the'] ,window_size 3
----------------------------------------
document: the cat in the hat
target: cat
context words ['the', 'in', 'the', 'hat'] ,window_size 3
----------------------------------------
document: the cat in the hat
target: in
context words ['the', 'cat', 'the', 'hat'] ,window_size 3
----------------------------------------
document: the cat in the hat
target: the
context words ['the', 'cat', 'in', 'hat'] ,window_size 3
----------------------------------------
document: the cat in the hat
target: hat
context words ['cat', 'in', 'the'] ,window_size 3
----------------------------------------
document: the quick brown fox
target: the
context words ['quick', 'brown', 'fox'] ,window_size 3
----------------------------------------
document: the quick brown fox
target: quick
context words ['the', 'brown', 'fox'] ,window_size 3
----------------------------------------
document: the quick brown fox
target: brown

In [17]:
training_pairs # this is the pairs of training formed the target and context word pairs according to context window size

[[('the', 'cat'),
  ('the', 'in'),
  ('the', 'the'),
  ('cat', 'the'),
  ('cat', 'in'),
  ('cat', 'the'),
  ('cat', 'hat'),
  ('in', 'the'),
  ('in', 'cat'),
  ('in', 'the'),
  ('in', 'hat'),
  ('the', 'the'),
  ('the', 'cat'),
  ('the', 'in'),
  ('the', 'hat'),
  ('hat', 'cat'),
  ('hat', 'in'),
  ('hat', 'the')],
 [('the', 'quick'),
  ('the', 'brown'),
  ('the', 'fox'),
  ('quick', 'the'),
  ('quick', 'brown'),
  ('quick', 'fox'),
  ('brown', 'the'),
  ('brown', 'quick'),
  ('brown', 'fox'),
  ('fox', 'the'),
  ('fox', 'quick'),
  ('fox', 'brown')],
 [('the', 'hat'),
  ('the', 'is'),
  ('the', 'blue'),
  ('hat', 'the'),
  ('hat', 'is'),
  ('hat', 'blue'),
  ('is', 'the'),
  ('is', 'hat'),
  ('is', 'blue'),
  ('blue', 'the'),
  ('blue', 'hat'),
  ('blue', 'is')]]

In [18]:
# Initialize word vectors randomly
embedding_dim = 10
learning_rate = 0.01
epochs = 500
word_vectors = []
vocab=[]
for d in Docs:
    vocab.append((list(set(d.lower().split(' ')))))
for v in vocab:
    word_vectors.append({word: np.random.rand(embedding_dim) for word in v})#initialize random values vector for each word 
    
for i in range(0,len(training_pairs)):
    # Train the Skip-gram model
    for epoch in range(epochs):
    
        for target_word, context_word in training_pairs[i]:
            # Forward pass
            input_vector = word_vectors[i][target_word]
            output_vector = word_vectors[i][context_word]

            # Calculate loss (using negative log likelihood)
            error = -np.log(np.exp(np.dot(input_vector, output_vector)))

            # Backward pass (update word vectors using gradient descent)
            gradient = input_vector * np.exp(np.dot(input_vector, output_vector)) / (1 + np.exp(np.dot(input_vector, output_vector)))
            word_vectors[i][target_word] -= learning_rate * gradient
            word_vectors[i][context_word] -= learning_rate * gradient

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Loss: {error}")
    print('-------------------------------')
    #word vectors
    for word, vector in word_vectors[i].items():
        print(f"Vector for '{word}': {vector}")
    print('-------------------------------')

Epoch 0, Loss: -2.004682748363519
-------------------------------
Vector for 'in': [-7.08549416e-04  1.40476584e-04 -3.43853366e-04 -5.81333419e-04
 -2.62263037e-04 -9.17635493e-04 -9.45590987e-05 -8.98408401e-04
  5.96805818e-04  6.73054005e-05]
Vector for 'the': [ 1.20387676e-04 -5.92906772e-05  1.04905803e-04  1.42348076e-04
  7.36244363e-05  1.98624754e-04  1.45755699e-05  2.08044778e-04
 -1.49273303e-04  7.99169337e-06]
Vector for 'cat': [-0.00030643  0.00020921 -0.00050576 -0.0005726  -0.00034617 -0.0007125
 -0.00011102 -0.00078546  0.00052242 -0.00016474]
Vector for 'hat': [ 0.0011613  -0.0003226   0.00095154  0.00129425  0.00068804  0.00184003
  0.00027668  0.00188781 -0.00120769  0.00012521]
-------------------------------
Epoch 0, Loss: -2.3569439229184677
-------------------------------
Vector for 'brown': [-0.00061899  0.00136345 -0.00162308  0.00312532  0.00117803 -0.00136243
 -0.00253807 -0.00217043 -0.00241094  0.00162926]
Vector for 'the': [-0.00218648 -0.00057473 -0.00

# TensorFlow Skip-gram

In [19]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dot, Dense,Flatten
from tensorflow.keras.models import Model
from nltk.tokenize import word_tokenize

# Create a vocabulary with unique words and indices
vocab = []
for doc in Docs:
    wrds=list(set(doc.lower().split()))
    vocab.append({w:wrds.index(w) for w in wrds})
reverse_vocab = []
for v in vocab:
    reverse_vocab.append({idx: word for word, idx in v.items()})

# Generate training pairs (target word, context word)
training=[]
for d,v in zip(Docs,vocab):
    training_pairs=[]
    for i, target_word in enumerate(d.lower().split(' ')):
        start = max(0, i - window_size)
        end = min(len(d.lower().split(' ')), i + window_size + 1)
        context_words = [d.lower().split(' ')[j] for j in range(start, end) if j != i]
        for context_word in context_words:
            training_pairs.append((v[target_word], v[context_word]))
    training.append(training_pairs)
    
    
# Define the Skip-gram model using TensorFlow
for i,v,t_p in zip(range(0,len(Docs)),vocab,training):
    target_word_input = tf.keras.layers.Input(shape=(1,), name="target_word")
    context_word_input = tf.keras.layers.Input(shape=(1,), name="context_word")

    embedding_layer = Embedding(input_dim=len(v), output_dim=embedding_dim)
    target_word_embedding = embedding_layer(target_word_input)#obtain respective embeddings
    context_word_embedding = embedding_layer(context_word_input)#obtain respective embeddings
    dot_product = Dot(axes=1, normalize=False)([target_word_embedding, context_word_embedding])#captures relations and similarity between context and target words
    #dot_product = Dot(axes=2)([target_word_embedding, context_word_embedding])
    output_layer = Dense(1, activation='sigmoid')(Flatten()(dot_product))  # Flatten the output

    model = Model(inputs=[target_word_input, context_word_input], outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy')
    
    
    # Train the Skip-gram model
    target_words = np.array([pair[0] for pair in t_p], dtype=np.int32)
    context_words = np.array([pair[1] for pair in t_p], dtype=np.int32)

    labels = np.array([1] * len(t_p), dtype=np.float32)# Positive labels for all training pairs
    
    model.fit({'target_word': target_words, 'context_word': context_words}, labels, epochs=500,verbose=0)

    # Access word vectors
    word_vectors = embedding_layer.get_weights()[0]
    for idx, word in reverse_vocab[i].items():
        print(f"Vector for '{word}': {word_vectors[idx]}")
    
    keras.utils.plot_model(model,show_shapes=True,show_layer_names=True)
    print("-----------------------------------------------------------")

Vector for 'in': [ 0.36872518  0.39219603  0.34225398  0.36937898 -0.35525295 -0.37965685
 -0.34941453 -0.41470155  0.3602024   0.3835445 ]
Vector for 'the': [ 0.3863065   0.37340966  0.39826486  0.40833113 -0.33449602 -0.3646308
 -0.38732684 -0.382091    0.3655319   0.33389097]
Vector for 'cat': [ 0.3513068   0.3460838   0.3494773   0.36490402 -0.4003787  -0.41152707
 -0.3724844  -0.3478648   0.3323159   0.40486374]
Vector for 'hat': [ 0.39758477  0.32779267  0.392022    0.3911184  -0.3826479  -0.4069367
 -0.3180633  -0.3647316   0.3549127   0.34646443]
-----------------------------------------------------------
Vector for 'brown': [ 0.36114398  0.34733352 -0.32631257  0.40080822  0.3642068   0.37763423
 -0.40822223  0.4071377   0.38608024 -0.36233202]
Vector for 'the': [ 0.39242572  0.38714066 -0.37875313  0.35951823  0.41999307  0.4028705
 -0.4069736   0.38205358  0.42476693 -0.3597098 ]
Vector for 'quick': [ 0.3949164   0.4136054  -0.45350263  0.34580472  0.3984888   0.38242403
 -0

# Latent Semantic Analysis (LSA)
LSA Main Components:
* **Term-Document Matrix:** LSA starts with the creation of a term-document matrix, where each row corresponds to a term (word) and each column corresponds to a document. The entries in the matrix represent the frequency of each term in each document.
* **Singular Value Decomposition (SVD):** SVD is a linear algebra technique used to factorize a matrix into three matrices: U, Σ (Sigma), and V^T (the transpose of V). In the context of LSA: U: Document-topic matrix (describes relationships between documents and topics), Σ (Sigma): Diagonal matrix containing singular values (describes the importance of each topic), & V^T: Term-topic matrix (describes relationships between terms and topics).
* **Dimensionality Reduction:** LSA typically involves reducing the dimensionality of the term-document matrix by retaining only the top singular values and corresponding columns in U, Σ, and V^T. This reduction helps capture the most significant semantic information while discarding noise and redundancy.
* **Semantic Space:** The reduced matrices U and V^T define a lower-dimensional semantic space where terms and documents are represented. Each document and term are mapped into this space, and their proximity in the space reflects semantic similarity.
* **Topic Modeling:** The columns of the U and V^T matrices represent topics in the semantic space. Documents and terms are associated with a linear combination of these topics. Topics capture the underlying themes or concepts in the corpus.

In [20]:
from sklearn.preprocessing import normalize

#Generate a document-term matrix of shape m x n having TF-IDF scores.
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(Docs)

# Convert the term-document matrix to a dense NumPy array
term_document_matrix = X.toarray()

# Normalize the term-document matrix
normalized_term_document_matrix = normalize(term_document_matrix, norm='l2', axis=1)

# Apply Latent Semantic Analysis (LSA) using Truncated SVD
def lsa(matrix, num_topics):
    # Perform Truncated SVD
    U, Sigma, Vt = np.linalg.svd(matrix, full_matrices=False)
    
    # Retain only the top 'num_topics' components
    U = U[:, :num_topics]
    Sigma = np.diag(Sigma[:num_topics])
    Vt = Vt[:num_topics, :]

    # Compute the document-topic matrix
    document_topic_matrix = U.dot(Sigma)
    
    # Compute the term-topic matrix
    term_topic_matrix = Vt.T
    
    return document_topic_matrix, term_topic_matrix

num_topics = 2
document_topic_matrix, term_topic_matrix = lsa(normalized_term_document_matrix, num_topics)

# Print the resulting matrices
print("Document-Topic Matrix:")
print(document_topic_matrix)

print("\nTerm-Topic Matrix:")
print(term_topic_matrix)

# Normalize each row to obtain probability-like scores
normalized_U = document_topic_matrix / np.sum(document_topic_matrix, axis=1, keepdims=True)

# Determine top topics for each document
top_topics_indices = np.argsort(normalized_U, axis=1)[:,::-1]
top_topics_scores = np.take_along_axis(normalized_U, top_topics_indices, axis=1)

# Print top topics for each document
print("\nTop Topics for Each Document:")
for i in range(normalized_U.shape[0]):
    print(f"Document {i+1}: Topics {top_topics_indices[i]} with Scores {top_topics_scores[i]}")

Document-Topic Matrix:
[[-0.86326717 -0.15662665]
 [-0.64932826  0.74819816]
 [-0.80299545 -0.43663456]]

Term-Topic Matrix:
[[-0.22161881 -0.28170621]
 [-0.17920818  0.48271962]
 [-0.18010249 -0.07638797]
 [-0.17920818  0.48271962]
 [-0.4017213  -0.35809418]
 [-0.18010249 -0.07638797]
 [-0.22161881 -0.28170621]
 [-0.17920818  0.48271962]
 [-0.76103198  0.04823747]]

Top Topics for Each Document:
Document 1: Topics [0 1] with Scores [0.84642847 0.15357153]
Document 2: Topics [1 0] with Scores [ 7.56750207 -6.56750207]
Document 3: Topics [0 1] with Scores [0.64777025 0.35222975]


In [21]:
# Alternatively, use sklearn's TruncatedSVD
#Generate a document-term matrix of shape m x n having TF-IDF scores.
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(Docs)

# Convert the term-document matrix to a dense NumPy array
term_document_matrix = X.toarray()

# Normalize the term-document matrix
normalized_term_document_matrix = normalize(term_document_matrix, norm='l2', axis=1)

def lsa_sklearn(matrix, num_topics):
    svd = TruncatedSVD(n_components=num_topics)
    svd.fit(matrix)
    U_sklearn = svd.transform(matrix)
    Sigma_sklearn = np.diag(svd.singular_values_)
    Vt_sklearn = svd.components_
    return U_sklearn, Sigma_sklearn, Vt_sklearn

num_topics=2
U_sklearn, Sigma_sklearn, Vt_sklearn = lsa_sklearn(term_document_matrix, num_topics)

# Print the resulting matrices using sklearn's TruncatedSVD
print("\nU (Document-topic matrix) - sklearn:")
print(U_sklearn)

print("\nSigma (Singular values) - sklearn:")
print(Sigma_sklearn)

print("\nVt (Term-topic matrix) - sklearn:")
print(Vt_sklearn)

# Determine top topics for each document
top_topics_indices = np.argsort(document_topic_matrix, axis=1)[:,::-1]
top_topics_scores = np.take_along_axis(document_topic_matrix, top_topics_indices, axis=1)

# Print top topics for each document
print("\nTop Topics for Each Document:")
for i in range(document_topic_matrix.shape[0]):
    print(f"Document {i+1}: Topics {top_topics_indices[i]} with Scores {top_topics_scores[i]}")


U (Document-topic matrix) - sklearn:
[[ 2.48703276 -0.37454454]
 [ 1.13180374  1.63007044]
 [ 1.50180299 -0.60821246]]

Sigma (Singular values) - sklearn:
[[3.11796791 0.        ]
 [0.         1.779701  ]]

Vt (Term-topic matrix) - sklearn:
[[ 0.15447908  0.11642007  0.2558222   0.11642007  0.41030128  0.2558222
   0.15447908  0.11642007  0.78254354]
 [-0.19202653  0.51465036 -0.11825224  0.51465036 -0.31027877 -0.11825224
  -0.19202653  0.51465036  0.08611936]]

Top Topics for Each Document:
Document 1: Topics [1 0] with Scores [-0.15662665 -0.86326717]
Document 2: Topics [1 0] with Scores [ 0.74819816 -0.64932826]
Document 3: Topics [1 0] with Scores [-0.43663456 -0.80299545]


# Word Embeddings using Neural Networks

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(Docs)
total_words = len(tokenizer.word_index) + 1

# Creating input sequences
input_sequences = tokenizer.texts_to_sequences(Docs)
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Define the neural network model
embedding_dim = 10
model = Sequential()
model.add(Embedding(total_words, embedding_dim, input_length=max_sequence_len))
model.add(Flatten())
#  used to convert the 2D array output from the Embedding layer into a 1D array, 
# which can then be fed into a Dense layer. The Embedding layer outputs a 3D tensor 
# with dimensions (batch_size, sequence_length, embedding_dimension). 
# The Flatten layer reshapes this tensor into a 2D tensor of shape 
# (batch_size, sequence_length * embedding_dimension).
model.add(Dense(1, activation='sigmoid'))
#The sigmoid activation function squashes the output between 0 and 1
# , which is suitable for binary classification problems where the output represents a probability.
# Compile the model
model.compile(optimizer='adam', loss='mse') # Using mean squared error (mse) as the loss for embeddings

# Train the model
model.fit(input_sequences, input_sequences, epochs=50, verbose=1)

Epoch 1/50
1/1 [==============================] - 1s 574ms/step - loss: 16.3568
Epoch 2/50
1/1 [==============================] - 0s 7ms/step - loss: 16.3426
Epoch 3/50
1/1 [==============================] - 0s 7ms/step - loss: 16.3285
Epoch 4/50
1/1 [==============================] - 0s 7ms/step - loss: 16.3144
Epoch 5/50
1/1 [==============================] - 0s 7ms/step - loss: 16.3002
Epoch 6/50
1/1 [==============================] - 0s 8ms/step - loss: 16.2861
Epoch 7/50
1/1 [==============================] - 0s 8ms/step - loss: 16.2720
Epoch 8/50
1/1 [==============================] - 0s 6ms/step - loss: 16.2579
Epoch 9/50
1/1 [==============================] - 0s 7ms/step - loss: 16.2437
Epoch 10/50
1/1 [==============================] - 0s 6ms/step - loss: 16.2296
Epoch 11/50
1/1 [==============================] - 0s 5ms/step - loss: 16.2154
Epoch 12/50
1/1 [==============================] - 0s 5ms/step - loss: 16.2013
Epoch 13/50
1/1 [==============================] - 0s 6ms/s

In [23]:
# Extracting word embeddings
word_embeddings = model.layers[0].get_weights()[0]  # Accessing weights of the Embedding layer

# Creating a new model to predict word embeddings
embedding_model = Sequential()
#  set trainable=False to prevent the model weights from being updated during inference.
embedding_model.add(Embedding(total_words, embedding_dim, weights=[word_embeddings], input_length=max_sequence_len, trainable=False))

# Compile the embedding model (not necessary if you're just using it for inference)
embedding_model.compile(optimizer='adam', loss='mse')  

# Example: Predicting word embedding for a specific word
word_index = tokenizer.word_index['cat']  # Assuming 'cat' is in the vocabulary
word_embedding = embedding_model.predict(np.array([[word_index]]))
print("Embedding for 'cat':", word_embedding)

1/1 [==============================] - 0s 102ms/step
Embedding for 'cat': [[[ 6.4948365e-02  3.1867530e-03 -1.4100554e-02 -7.9918794e-02
    8.4939704e-05 -3.2026764e-02  2.8976372e-02 -3.4671575e-02
    1.5345849e-02  8.4831215e-02]]]


# Word Embeddings using Bert (Transformers)

In [24]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')


# Tokenize text
tokenized_texts = tokenizer(Docs, return_tensors='tf', padding=True, truncation=True, max_length=128)

# Obtain word embeddings
output = model(**tokenized_texts)

# Extract word embeddings
word_embeddings = output.last_hidden_state.numpy()

# Calculate mean word embeddings for each sentence
mean_word_embeddings = np.mean(word_embeddings, axis=1)

# Display mean word embeddings for each sentence
for i, mean_embedding in enumerate(mean_word_embeddings):
    print(f"Mean Word Embeddings for Sentence {i + 1}:")
    print(mean_embedding)
    print('----------')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Mean Word Embeddings for Sentence 1:
[-2.04305381e-01 -1.81919500e-01 -5.67251980e-01 -2.50381410e-01
  5.18147424e-02 -8.39290097e-02 -1.64983734e-01  5.50135434e-01
 -3.02776873e-01  1.82250157e-01  4.48545367e-01 -3.83129060e-01
 -1.01198964e-01  1.55241579e-01 -5.67099094e-01 -7.92787895e-02
  2.81859905e-01  1.31031290e-01 -4.40042585e-01  8.52416232e-02
  1.05672017e-01  2.95385778e-01 -2.56191880e-01  3.67898762e-01
  1.41279429e-01  1.56280980e-01 -2.50345677e-01  2.29516372e-01
  1.50050908e-01 -4.32113931e-02  2.49939680e-01 -2.33981475e-01
  3.00897323e-02 -1.76527426e-01 -2.66757905e-01  7.28175044e-02
 -1.68152958e-01  7.67114833e-02 -4.35148925e-01  1.76707610e-01
 -4.86129671e-01 -1.72616959e-01  7.87293538e-02  1.26926482e-01
 -5.82963899e-02 -2.12164372e-01  2.98652917e-01 -1.09550789e-01
  4.24578488e-01 -2.75301278e-01 -2.54252672e-01  4.35968265e-02
  6.24317899e-02 -1.11223452e-01  3.73741210e-01  7.75230110e-01
  3.12518835e-01 -2.19001517e-01 -3.67718264e-02 -2.1

* **Count of Words (Bag-of-Words Model):**  Early NLP models represented documents using a Bag-of-Words (BoW) model, which counts the occurrence of words in a document without considering their order. This approach provides a basic representation of documents but lacks capturing semantic relationships.
* **Term Frequency-Inverse Document Frequency (TF-IDF):** introduced a weighting scheme that considers not only the count of words in a document but also their importance in the entire corpus. It helps identify words that are significant to a particular document but not frequent across all documents.
* **Latent Semantic Analysis (LSA):** also known as Latent Semantic Indexing (LSI), applies singular value decomposition (SVD) to the term-document matrix. It reduces the dimensionality of the space and captures latent semantic relationships between words and documents, improving representation.
* **Skip-gram and Continuous Bag of Words (CBOW):** Word2Vec, introduced by Mikolov et al., includes two models: Skip-gram and Continuous Bag of Words (CBOW). These models use shallow neural networks to learn word embeddings. Skip-gram predicts context words given a target word, while CBOW predicts a target word given its context. Word2Vec significantly improves word embeddings' quality and captures semantic relationships.
* **Global Vectors for Word Representation (GloVe):** introduces a global approach by training on aggregated global word co-occurrence statistics. It leverages a matrix factorization technique to capture the relationships between words in a more efficient manner, producing high-quality word embeddings.
* **FastText and Subword Embeddings:** also by Mikolov et al., extends word embeddings to subword level. It represents words as bags of character n-grams, enabling the generation of embeddings for out-of-vocabulary words and capturing morphological information.
* **Transformer Architecture:** introduced by Vaswani et al., revolutionizes NLP by employing self-attention mechanisms. It enables models like BERT (Bidirectional Encoder Representations from Transformers) to learn contextualized word embeddings, considering the entire input sequence bidirectionally.
* **Contextualized Word Embeddings (BERT, GPT, ELMo):** BERT, GPT (Generative Pre-trained Transformer), and ELMo (Embeddings from Language Models) introduce contextualized embeddings by considering surrounding words and contexts. These models capture rich contextual information, leading to state-of-the-art performance in various NLP tasks.
* **Transfer Learning and Fine-Tuning:** Pre-trained language models, such as BERT and GPT, can be fine-tuned on specific downstream tasks. This transfer learning approach significantly reduces the need for large labeled datasets and improves performance on specific tasks.
* **Multimodal Embeddings:** are extended beyond text to include multimodal information, combining textual and visual features. Models like CLIP (Contrastive Language-Image Pre-training) learn joint representations of text and images, enabling cross-modal understanding.


**Key breakpoints:**

* **From word counts to context:** Moving beyond simple frequency to considering surrounding words for richer representations.
* **Pre-training on large corpora:** Utilizing massive amounts of text data to learn general language understanding.
* **Bi-directional and attention mechanisms:** Capturing complex relationships between words in a sentence.
* **Transformers and self-attention:** Enabling efficient learning of long-range dependencies.
* **LLMs reaching human-level performance in certain tasks:** Highlighting the potential of language models for natural communication and problem-solving.

**Future directions:**

* **Explainability and interpretability:** Understanding how LLMs work and make decisions.
* **Addressing biases and fairness:** Ensuring models are inclusive and represent diverse perspectives.
* **Combining symbolic and neural approaches:** Integrating logic and reasoning with language understanding.
